In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
pd.set_option('max_columns', None)
pd.set_option('display.width', 1000)
from prepare_datasets import *
from sklearn.model_selection import train_test_split
from create_model import CreateModel, MLPClassifier
from user_preferences import userPreferences
from care_explainer import CAREExplainer
from wachter_explainer import WACHTERExplainer
from generate_text_explanations import GenerateTextExplanations
from utils import *
import numpy as np
np.set_printoptions(threshold=np.inf)

In [ ]:
# reading a data set
dataset = PrepareAdult('./datasets/','adult.csv')
# splitting the data set into train and test sets
X, y = dataset['X_ord'], dataset['y']
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#blackbox model
blackbox = CreateModel(dataset, X_train, X_test, Y_train, Y_test, 'classification', 'nn-c', MLPClassifier)
predict_fn = lambda x: blackbox.predict_classes(x).ravel()
predict_proba_fn = lambda x: np.asarray([1-blackbox.predict(x).ravel(), blackbox.predict(x).ravel()]).transpose()

In [ ]:
#select random set of negative outcome (<50k) 
outcome = blackbox.predict_classes(ord2ohe(X_test, dataset)).ravel()
outcome = pd.DataFrame(outcome)
outcome.rename(columns={outcome.columns[0]: "Outcome" }, inplace = True)
negative_outcome = outcome[outcome['Outcome']==0]
sample = negative_outcome.sample(n=5, random_state=123)
sample = sample.index.values.tolist() 

In [ ]:
for i in sample:
    x_ord = X_test[i]
    CARE_output = CAREExplainer(x_ord, X_train, Y_train, dataset, 'classification', predict_fn, predict_proba_fn,
                                        SOUNDNESS=True, COHERENCY=True, ACTIONABILITY=True,
                                        user_preferences=userPreferences(dataset, x_ord), cf_class='opposite',
                                        probability_thresh=0.5, n_cf=5)
    
    WACHTER_output = WACHTERExplainer(x_ord, X_train, Y_train, dataset, 'classification', predict_fn, predict_proba_fn,
                                        SOUNDNESS=False, COHERENCY=False, ACTIONABILITY=False,
                                        user_preferences=userPreferences(dataset, x_ord), cf_class='opposite',
                                        probability_thresh=0.5, n_cf=1)

    exec(f"CARE{i} = \
    pd.merge(CARE_output['x_cfs_highlight'],\
    CARE_output['x_cfs_eval'][['Class']], \
    left_index=True, right_index=True, how='left')")

    exec(f"WACHTER{i} = \
    pd.merge(WACHTER_output['x_cfs_highlight'],\
    WACHTER_output['x_cfs_eval'][['Class']], \
    left_index=True, right_index=True, how='left')")


In [ ]:
for i, j in zip(range(1,5), sample):
    exec(f"CARE{i} = CARE{j}")

for i, j in zip(range(1,5), sample):
    exec(f"WACHTER{i} = WACHTER{j}")

CARE_adult = pd.concat([CARE0, CARE1, CARE2, CARE3, CARE4, CARE5, CARE6, CARE7, CARE8, CARE9, CARE10, CARE11, CARE12, CARE13, CARE14, CARE15, CARE16, CARE17, CARE18, CARE19], ignore_index=False)
WACHTER_adult = pd.concat([WACHTER0, WACHTER1, WACHTER2, WACHTER3, WACHTER4, WACHTER5, WACHTER6, WACHTER7, WACHTER8, WACHTER9, WACHTER10, WACHTER12, WACHTER13, WACHTER14, WACHTER15, WACHTER16, WACHTER17, WACHTER18, WACHTER19], ignore_index=False)
